# Import packages

In [1]:
import pandas as pd

In [2]:
import plotly
import plotly.express as px

import datapane as dp
plotly.offline.init_notebook_mode(connected=True)

In [3]:
import re
import pickle


from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Upload json files

In [4]:
files= ['0.json', '1.json', '2.json', '3.json', '4.json','42.json', '15693.json']

# create an empty list to store the DataFrames
dfsList = [pd.read_json(file, orient='records') for file in files]

# combine all DataFrames into a single DataFrame
crossrefDF = pd.concat(dfsList, ignore_index=True)


In [5]:
crossrefDF

,items
0,"{'URL': 'http://dx.doi.org/10.1021/ac020069k',..."
1,{'URL': 'http://dx.doi.org/10.1103/physrevc.65...
2,{'abstract': '<jats:p> <jats:bold>...
3,{'URL': 'http://dx.doi.org/10.1093/jjco/hyf035...
4,{'abstract': '<jats:p>The integration of the I...
...,...
34995,"{'URL': 'http://dx.doi.org/10.5414/prp22125', ..."
34996,{'URL': 'http://dx.doi.org/10.4172/2161-1122.1...
34997,{'URL': 'http://dx.doi.org/10.1371/journal.pon...
34998,{'URL': 'http://dx.doi.org/10.1371/journal.pon...


In [6]:
noAuthors = [i for i in range(len(crossrefDF)) if 'author' not in crossrefDF['items'][i]]

Authors = [i for i in range(len(crossrefDF)) if 'author'  in crossrefDF['items'][i]]

In [7]:
len(noAuthors) + len(Authors) == len(crossrefDF)

True

## take only rows with authors

In [25]:
crossrefAuth = crossrefDF.iloc[Authors].copy()

crossrefAuth.reset_index(inplace= True)
crossrefAuth.drop(columns = ['index'], inplace = True)


In [26]:
crossrefAuth

,items
0,"{'URL': 'http://dx.doi.org/10.1021/ac020069k',..."
1,{'URL': 'http://dx.doi.org/10.1103/physrevc.65...
2,{'abstract': '<jats:p> <jats:bold>...
3,{'URL': 'http://dx.doi.org/10.1093/jjco/hyf035...
4,{'abstract': '<jats:p>The integration of the I...
...,...
32175,{'URL': 'http://dx.doi.org/10.5354/0719-3769.2...
32176,"{'URL': 'http://dx.doi.org/10.1136/bmj.g2655',..."
32177,"{'URL': 'http://dx.doi.org/10.5414/prp22125', ..."
32178,{'URL': 'http://dx.doi.org/10.4172/2161-1122.1...


## extract 'DOI'

In [27]:
crossrefAuth.loc[:, 'DOI'] = crossrefAuth['items'].apply(lambda x: x['DOI'])

In [28]:
crossrefAuth.head()

,items,DOI
0,"{'URL': 'http://dx.doi.org/10.1021/ac020069k',...",10.1021/ac020069k
1,{'URL': 'http://dx.doi.org/10.1103/physrevc.65...,10.1103/physrevc.65.064904
2,{'abstract': '<jats:p> <jats:bold>...,10.1161/01.cir.0000013846.72805.7e
3,{'URL': 'http://dx.doi.org/10.1093/jjco/hyf035...,10.1093/jjco/hyf035
4,{'abstract': '<jats:p>The integration of the I...,10.1108/14684520010357338


## extract 'authors' --- number of authors

In [29]:
crossrefAuth.loc[:,'authors'] = crossrefAuth['items'].apply(lambda x: x['author'])


In [30]:
crossrefAuth.head()

,items,DOI,authors
0,"{'URL': 'http://dx.doi.org/10.1021/ac020069k',...",10.1021/ac020069k,"[{'given': 'Gi Hun', 'family': 'Seong', 'seque..."
1,{'URL': 'http://dx.doi.org/10.1103/physrevc.65...,10.1103/physrevc.65.064904,"[{'given': 'S. V.', 'family': 'Akkelin', 'sequ..."
2,{'abstract': '<jats:p> <jats:bold>...,10.1161/01.cir.0000013846.72805.7e,"[{'given': 'Dinesh K.', 'family': 'Kalra', 'se..."
3,{'URL': 'http://dx.doi.org/10.1093/jjco/hyf035...,10.1093/jjco/hyf035,"[{'given': 'E.', 'family': 'Bakan', 'sequence'..."
4,{'abstract': '<jats:p>The integration of the I...,10.1108/14684520010357338,"[{'given': 'Kaba', 'family': 'Abdoulaye', 'seq..."


In [32]:
numAuthors = [len(crossrefAuth.iloc[i]['authors']) for i in range(len(crossrefAuth))]

In [33]:
## yparxoun lathi  ---> kalytera number of affiliations
crossrefAuth.loc[:,'# authors'] = numAuthors

In [34]:
crossrefAuth.head()

,items,DOI,authors,# authors
0,"{'URL': 'http://dx.doi.org/10.1021/ac020069k',...",10.1021/ac020069k,"[{'given': 'Gi Hun', 'family': 'Seong', 'seque...",3
1,{'URL': 'http://dx.doi.org/10.1103/physrevc.65...,10.1103/physrevc.65.064904,"[{'given': 'S. V.', 'family': 'Akkelin', 'sequ...",3
2,{'abstract': '<jats:p> <jats:bold>...,10.1161/01.cir.0000013846.72805.7e,"[{'given': 'Dinesh K.', 'family': 'Kalra', 'se...",10
3,{'URL': 'http://dx.doi.org/10.1093/jjco/hyf035...,10.1093/jjco/hyf035,"[{'given': 'E.', 'family': 'Bakan', 'sequence'...",1
4,{'abstract': '<jats:p>The integration of the I...,10.1108/14684520010357338,"[{'given': 'Kaba', 'family': 'Abdoulaye', 'seq...",2


## extract 'affiliations' --- number of affiliations

In [37]:
def getAff(k):
   return [crossrefAuth['authors'][k][j]['affiliation'] for j in range(len(crossrefAuth['authors'][k]))]
    

In [38]:
Affiliations = [getAff(k) for k in range(len(crossrefAuth))]

crossrefAuth.loc[:,'affiliations'] = Affiliations

In [39]:
len(Affiliations[2])

10

In [40]:
numAffil = [len(Affiliations[i]) for i in range(len(crossrefAuth))]

In [41]:
crossrefAuth.loc[:,'# Affil'] = numAffil

In [42]:
crossrefAuth.head()

,items,DOI,authors,# authors,affiliations,# Affil
0,"{'URL': 'http://dx.doi.org/10.1021/ac020069k',...",10.1021/ac020069k,"[{'given': 'Gi Hun', 'family': 'Seong', 'seque...",3,"[[{'name': 'Department of Chemistry, Texas A&M...",3
1,{'URL': 'http://dx.doi.org/10.1103/physrevc.65...,10.1103/physrevc.65.064904,"[{'given': 'S. V.', 'family': 'Akkelin', 'sequ...",3,"[[], [], []]",3
2,{'abstract': '<jats:p> <jats:bold>...,10.1161/01.cir.0000013846.72805.7e,"[{'given': 'Dinesh K.', 'family': 'Kalra', 'se...",10,[[{'name': 'From the Section of Cardiology (D....,10
3,{'URL': 'http://dx.doi.org/10.1093/jjco/hyf035...,10.1093/jjco/hyf035,"[{'given': 'E.', 'family': 'Bakan', 'sequence'...",1,[[]],1
4,{'abstract': '<jats:p>The integration of the I...,10.1108/14684520010357338,"[{'given': 'Kaba', 'family': 'Abdoulaye', 'seq...",2,"[[], []]",2


## Clean 'empty' affiliations

In [43]:
possibleEmptyAff = []

for k in range(len(crossrefAuth)):
    if len(crossrefAuth['affiliations'][k][0]) == 0:
        possibleEmptyAff.append(k)

In [44]:
len(possibleEmptyAff)

24908

In [45]:
crossrefAuth['affiliations'].iloc[2611]

[[],
 [],
 [],
 [],
 [{'name': 'Department of Organic and Polymeric Materials, Tokyo Institute of Technology, 2-12-1, Ohokayama, Meguro-ku, Tokyo 152-8552, Japan'}]]

In [46]:
nonEmptyAff = []

for k in possibleEmptyAff:
    for j in range(len(crossrefAuth['affiliations'].iloc[k])):
        if len(crossrefAuth['affiliations'].iloc[k][j]) != 0:
            nonEmptyAff.append(k)
    
    

In [47]:

FinalEmptyyAff=  [x for x in possibleEmptyAff if x not in nonEmptyAff] 

In [48]:
FinalNonEmptyAff = [x for x in range(len(crossrefAuth)) if x not in FinalEmptyyAff]

# affilDF: crossrefAuth subdataframe with nonpempty affiliation lists

In [49]:
affilDF = crossrefAuth.iloc[FinalNonEmptyAff].copy()
affilDF.reset_index(inplace = True)
affilDF.drop(columns = ['index'], inplace = True)

## still some cleaning: cases with empty brackets [{}]

In [50]:
affilDF[affilDF['DOI'] == '10.48130/emst-2022-0020']

,items,DOI,authors,# authors,affiliations,# Affil
6117,{'URL': 'http://dx.doi.org/10.48130/emst-2022-...,10.48130/emst-2022-0020,"[{'given': 'Xiangyu (Dale)', 'family': 'Li', '...",4,"[[{}], [], [], []]",4


In [51]:
for k in range(len(affilDF)):
    if len(affilDF['affiliations'][k][0]) != 0 and affilDF['affiliations'][k][0][0] == {}:
        print(k)

6117
6118


In [52]:
emptyBrackets = [k for k in range(len(affilDF)) if len(affilDF['affiliations'][k][0]) != 0 and affilDF['affiliations'][k][0][0] == {}]

In [53]:
affilDF.iloc[emptyBrackets]

,items,DOI,authors,# authors,affiliations,# Affil
6117,{'URL': 'http://dx.doi.org/10.48130/emst-2022-...,10.48130/emst-2022-0020,"[{'given': 'Xiangyu (Dale)', 'family': 'Li', '...",4,"[[{}], [], [], []]",4
6118,{'URL': 'http://dx.doi.org/10.48130/emst-2022-...,10.48130/emst-2022-0018,"[{'given': 'Fan', 'family': 'Yang', 'sequence'...",2,"[[{}], []]",2


In [54]:
affilDF.copy()

,items,DOI,authors,# authors,affiliations,# Affil
0,"{'URL': 'http://dx.doi.org/10.1021/ac020069k',...",10.1021/ac020069k,"[{'given': 'Gi Hun', 'family': 'Seong', 'seque...",3,"[[{'name': 'Department of Chemistry, Texas A&M...",3
1,{'abstract': '<jats:p> <jats:bold>...,10.1161/01.cir.0000013846.72805.7e,"[{'given': 'Dinesh K.', 'family': 'Kalra', 'se...",10,[[{'name': 'From the Section of Cardiology (D....,10
2,{'abstract': '<jats:p>Hyperhomocysteinemia (hH...,10.1161/hy02t2.102992,"[{'given': 'Ningjun', 'family': 'Li', 'sequenc...",3,"[[{'name': 'From the Department of Physiology,...",3
3,{'abstract': '<jats:p> The nuclear...,10.1126/science.1073633,"[{'given': 'Jianping', 'family': 'Hu', 'sequen...",6,"[[{'name': 'Plant Biology Laboratory,'}, {'nam...",6
4,{'URL': 'http://dx.doi.org/10.1089/10872910260...,10.1089/10872910260066679,"[{'given': 'Edward E.', 'family': 'Eaton', 'se...",5,[[{'name': 'Albert Einstein College of Medicin...,5
...,...,...,...,...,...,...
7294,{'abstract': '<jats:p>Applications of remote s...,10.1098/rstb.2013.0197,"[{'given': 'Véronique', 'family': 'St-Louis', ...",9,[[{'name': 'Department of Forest and Wildlife ...,9
7295,"{'URL': 'http://dx.doi.org/10.1002/jcb.24822',...",10.1002/jcb.24822,"[{'given': 'Nahid', 'family': 'Davoodian', 'se...",4,[[{'name': 'Department of Clinical Biochemistr...,4
7296,{'URL': 'http://dx.doi.org/10.1049/mnl.2013.07...,10.1049/mnl.2013.0750,"[{'given': 'Huicong', 'family': 'Liu', 'sequen...",5,[[{'name': 'Robotics and Microsystems Center a...,5
7297,{'URL': 'http://dx.doi.org/10.1002/jbm.a.35198...,10.1002/jbm.a.35198,"[{'given': 'Alok', 'family': 'Kumar', 'sequenc...",3,[[{'name': 'Laboratory for Biomaterials; Mater...,3


In [55]:
affilDF.drop(emptyBrackets, inplace = True)

In [56]:
affilDF.reset_index(inplace = True)

In [57]:
affilDF.copy()

,index,items,DOI,authors,# authors,affiliations,# Affil
0,0,"{'URL': 'http://dx.doi.org/10.1021/ac020069k',...",10.1021/ac020069k,"[{'given': 'Gi Hun', 'family': 'Seong', 'seque...",3,"[[{'name': 'Department of Chemistry, Texas A&M...",3
1,1,{'abstract': '<jats:p> <jats:bold>...,10.1161/01.cir.0000013846.72805.7e,"[{'given': 'Dinesh K.', 'family': 'Kalra', 'se...",10,[[{'name': 'From the Section of Cardiology (D....,10
2,2,{'abstract': '<jats:p>Hyperhomocysteinemia (hH...,10.1161/hy02t2.102992,"[{'given': 'Ningjun', 'family': 'Li', 'sequenc...",3,"[[{'name': 'From the Department of Physiology,...",3
3,3,{'abstract': '<jats:p> The nuclear...,10.1126/science.1073633,"[{'given': 'Jianping', 'family': 'Hu', 'sequen...",6,"[[{'name': 'Plant Biology Laboratory,'}, {'nam...",6
4,4,{'URL': 'http://dx.doi.org/10.1089/10872910260...,10.1089/10872910260066679,"[{'given': 'Edward E.', 'family': 'Eaton', 'se...",5,[[{'name': 'Albert Einstein College of Medicin...,5
...,...,...,...,...,...,...,...
7292,7294,{'abstract': '<jats:p>Applications of remote s...,10.1098/rstb.2013.0197,"[{'given': 'Véronique', 'family': 'St-Louis', ...",9,[[{'name': 'Department of Forest and Wildlife ...,9
7293,7295,"{'URL': 'http://dx.doi.org/10.1002/jcb.24822',...",10.1002/jcb.24822,"[{'given': 'Nahid', 'family': 'Davoodian', 'se...",4,[[{'name': 'Department of Clinical Biochemistr...,4
7294,7296,{'URL': 'http://dx.doi.org/10.1049/mnl.2013.07...,10.1049/mnl.2013.0750,"[{'given': 'Huicong', 'family': 'Liu', 'sequen...",5,[[{'name': 'Robotics and Microsystems Center a...,5
7295,7297,{'URL': 'http://dx.doi.org/10.1002/jbm.a.35198...,10.1002/jbm.a.35198,"[{'given': 'Alok', 'family': 'Kumar', 'sequenc...",3,[[{'name': 'Laboratory for Biomaterials; Mater...,3


In [58]:
affilDF.drop(columns = ['index'], inplace = True)

# Cleaning affiliations step by step

## 1. "Unique" affiliations --- number of unique affiliations

In [64]:
uniqueAff = [] 

for i in range(len(affilDF)):
    uniqueAff.append(list(set([x[0] for x in [list(d.values()) for d in  [item for sublist in affilDF['affiliations'].iloc[i] for item in sublist]]])))


In [65]:
affilDF.loc[:,'uniqueAff'] = uniqueAff

In [66]:
numUniqueAff = [len(affilDF['uniqueAff'].iloc[i]) for i in range(len(affilDF))]

In [67]:
affilDF.loc[:,'# uniqueAff'] = numUniqueAff

In [68]:
affilDF.head()

,items,DOI,authors,# authors,affiliations,# Affil,uniqueAff,# uniqueAff
0,"{'URL': 'http://dx.doi.org/10.1021/ac020069k',...",10.1021/ac020069k,"[{'given': 'Gi Hun', 'family': 'Seong', 'seque...",3,"[[{'name': 'Department of Chemistry, Texas A&M...",3,"[Department of Chemistry, Texas A&M University...",1
1,{'abstract': '<jats:p> <jats:bold>...,10.1161/01.cir.0000013846.72805.7e,"[{'given': 'Dinesh K.', 'family': 'Kalra', 'se...",10,[[{'name': 'From the Section of Cardiology (D....,10,"[From the Section of Cardiology (D.K.K., X.Z.,...",1
2,{'abstract': '<jats:p>Hyperhomocysteinemia (hH...,10.1161/hy02t2.102992,"[{'given': 'Ningjun', 'family': 'Li', 'sequenc...",3,"[[{'name': 'From the Department of Physiology,...",3,"[From the Department of Physiology, Medical Co...",1
3,{'abstract': '<jats:p> The nuclear...,10.1126/science.1073633,"[{'given': 'Jianping', 'family': 'Hu', 'sequen...",6,"[[{'name': 'Plant Biology Laboratory,'}, {'nam...",6,"[Plant Biology Laboratory,, Laboratory of Neur...",3
4,{'URL': 'http://dx.doi.org/10.1089/10872910260...,10.1089/10872910260066679,"[{'given': 'Edward E.', 'family': 'Eaton', 'se...",5,[[{'name': 'Albert Einstein College of Medicin...,5,"[Albert Einstein College of Medicine, Bronx, N...",4


## 2. Remove stop words ['from', 'the']

In [69]:
stopWords = ['from', 'the', 'From', 'The']

In [70]:
def remove_stop_words(text):
    words = text.split()
    filtered_words = [word for word in words if word not in stopWords]
    return ' '.join(filtered_words)


# apply the function to the column  affilDF['uniqueAff'] to create column affilDF.loc[:,'uniqueAff1']

affilDF.loc[:,'uniqueAff1'] = affilDF['uniqueAff'].apply(lambda x: [remove_stop_words(s) for s in x])


## 3. Remove parenthesis 

In [71]:
def remove_parentheses(text):
   return re.sub(r'\([^()]*\)', '', text)

# apply the function to each list element of column affilDF['uniqueAff1'] to remove substrings inside parentheses

affilDF.loc[:,'uniqueAff1'] = affilDF['uniqueAff1'].apply(lambda x: [remove_parentheses(s) for s in x])


## 4. Remove @#$%characters 

In [72]:
affNoSymbols = []

for i in range(len(list(affilDF['uniqueAff1']))):
    L = list(affilDF['uniqueAff1'])[i]
    for i in range(len(L)):
        L[i] = re.sub(r'[^\w\s,Α-Ωα-ωぁ-んァ-ン一-龯，]', '', L[i])
    affNoSymbols.append(L)


In [73]:
affNoSymbols = [[item for item in inner_list if item != "inc"] for inner_list in affNoSymbols]


In [74]:
affilDF['uniqueAff1'] = affNoSymbols


## 5. check 'sub'-affiliations (affiliations that are contained in other affiliations of the same DOI)

In [79]:
newAff0 = []

for k in range(len(affilDF)):
    
    L2 = []
    for s1 in affilDF['uniqueAff1'].iloc[k]:
        is_substring = False
        for s2 in affilDF['uniqueAff1'].iloc[k]:
            if s1 != s2 and s1 in s2:
                is_substring = True
                break
        if not is_substring:
            L2.append(s1)
    newAff0.append(L2)

In [80]:
newAffList = [list(set(newAff0[k])) for k in range(len(newAff0))]

In [82]:
affilDF['uniqueAff2'] = newAffList

In [83]:
affilDF.head(3)

,items,DOI,authors,# authors,affiliations,# Affil,uniqueAff,# uniqueAff,uniqueAff1,uniqueAff2
0,"{'URL': 'http://dx.doi.org/10.1021/ac020069k',...",10.1021/ac020069k,"[{'given': 'Gi Hun', 'family': 'Seong', 'seque...",3,"[[{'name': 'Department of Chemistry, Texas A&M...",3,"[Department of Chemistry, Texas A&M University...",1,"[Department of Chemistry, Texas AM University,...","[Department of Chemistry, Texas AM University,..."
1,{'abstract': '<jats:p> <jats:bold>...,10.1161/01.cir.0000013846.72805.7e,"[{'given': 'Dinesh K.', 'family': 'Kalra', 'se...",10,[[{'name': 'From the Section of Cardiology (D....,10,"[From the Section of Cardiology (D.K.K., X.Z.,...",1,[Section of Cardiology and Echocardiography L...,[Section of Cardiology and Echocardiography L...
2,{'abstract': '<jats:p>Hyperhomocysteinemia (hH...,10.1161/hy02t2.102992,"[{'given': 'Ningjun', 'family': 'Li', 'sequenc...",3,"[[{'name': 'From the Department of Physiology,...",3,"[From the Department of Physiology, Medical Co...",1,"[Department of Physiology, Medical College of ...","[Department of Physiology, Medical College of ..."


## 6. Split strings where ',' or ';' appears

In [84]:
newAffkomma = []

for k in range(len(affilDF)):
    
    new_list = []
    for item in affilDF['uniqueAff2'].iloc[k]:
        for s in re.split(r'[;,]', item):
            new_list.append(s.strip())

    newAffkomma.append(new_list)



In [85]:
for i in range(len(newAffkomma)):
        j= 0
        while j < len(newAffkomma[i]):
                if len(newAffkomma[i][j]) > 0 and newAffkomma[i][j][-1] in [',','.',' ']:
                        newAffkomma[i][j] = newAffkomma[i][j][:-1]
                        
                if len(newAffkomma[i][j]) > 1 and newAffkomma[i][j][0] == ' ' and newAffkomma[i][j][1] == ' ' : 
                        newAffkomma[i][j] = newAffkomma[i][j][2:] 
                                    
                elif len(newAffkomma[i][j]) > 0 and newAffkomma[i][j][0] == ' ': 
                        newAffkomma[i][j] = newAffkomma[i][j][1:] 
                      


                j = j+1

In [86]:
newAffkomma_1 = [[s for s in lst if s != ''] for lst in newAffkomma]


In [87]:
affilDF.loc[:,'uniqueAff4'] = newAffkomma_1

## 7. lower()

In [88]:
removeList = ['university','research institute','laboratory' , 'universit','gmbh', 'inc', 'university of', 'research center', 
'university college','national institute of', 'school of medicine', "university school", 'graduate school of', 'graduate school of engineering', 
'institute of tropical medicine', 'institute of virology', 'faculty of medicine','laboratory']

In [89]:
uniqueAff4_low = [[x.lower() for x in affilDF['uniqueAff4'].iloc[i] if x.lower() not in  removeList] for i in range(len(affilDF))] 

In [90]:
affilDF['uniqueAff4'] = uniqueAff4_low


# Labels based on legalnames of openAIRE's organizations

In [91]:
uniList = ['institu', 'istituto','univ', 'college', 'center', 'centre' , 'cnrs', 'faculty','school' , 'academy' , 'école', 'hochschule' , 'ecole' ]

labList = ['lab']

hosplList = ['hospital' ,'clinic', 'hôpital']

gmbhList = ['gmbh', 'company' , 'industr', 'etaireia' , 'corporation', 'inc']

musList =  ['museum', 'library']

foundList =  ['foundation' , 'association','organization' ,'society', 'group' ]

deptList = ['district' , 'federation'  , 'government' , 'municipal' , 'county', 'ministry','council', 'agency']

unknownList = ['unknown']

#######   Dictionaries ##########

uniDict = {k: 'Univ/Inst' for k in uniList}   

labDict = {k: 'Laboratory' for k in labList} 

hosplDict = {k: 'Hospital' for k in hosplList}   

gmbhDict = {k: 'Company' for k in gmbhList}   

musDict = {k: 'Museum' for k in musList}   

#schoolDict = {k: 'School' for k in schoolList}   

foundDict = {k: 'Foundation' for k in foundList}   

deptDict = {k: 'Government' for k in deptList}   

unknownDict =  {k: 'Unknown' for k in unknownList}   

categDictsList = [uniDict, labDict, hosplDict, gmbhDict, musDict, #schoolDict, 
                  foundDict, deptDict, unknownDict]

################# Final Dictionary #####################

categDicts = {}
i = 0
while i in range(len(categDictsList)):
    categDicts.update(categDictsList[i])
    i = i+1
    
    


## affiliationsDict

In [92]:
affiliationsDict = {}

for i in range(len(affilDF)):
    affiliationsDict[i] = affilDF['uniqueAff4'].iloc[i]
    


In [93]:
d_new = {}

# iterate over the keys of affiliationsDict
for k in range(len(affiliationsDict)):
    # get the list associated with the current key in affiliationsDict
    L = affiliationsDict.get(k, [])
    
    # apply the mapping using a list comprehension
    mapped_list = [[s, v] for s in L for k2, v in categDicts.items() if k2 in s]
    
    # add the mapped list to the new dictionary d_new
    d_new[k] = mapped_list

In [94]:
affilDF['Dictionary'] = d_new

## what is not included 

In [95]:
notInList = [i for i in range(len(affilDF)) if len(affilDF['Dictionary'].iloc[i]) == 0]
    

In [96]:
affilDF.iloc[notInList]

,items,DOI,authors,# authors,affiliations,# Affil,uniqueAff,# uniqueAff,uniqueAff1,uniqueAff2,uniqueAff4,Dictionary
5,{'URL': 'http://dx.doi.org/10.1126/science.296...,10.1126/science.296.5576.2123a,"[{'given': 'Wayne', 'family': 'Kondro', 'seque...",1,[[{'name': 'Wayne Kondro writes from Ottawa.'}]],1,[Wayne Kondro writes from Ottawa.],1,[Wayne Kondro writes Ottawa],[Wayne Kondro writes Ottawa],[wayne kondro writes ottawa],[]
6,{'URL': 'http://dx.doi.org/10.1046/j.1440-1665...,10.1046/j.1440-1665.2002.t01-1-00428.x,"[{'given': 'Bob', 'family': 'Adler', 'sequence...",1,"[[{'name': 'Melbourne, Victoria'}]]",1,"[Melbourne, Victoria]",1,"[Melbourne, Victoria]","[Melbourne, Victoria]","[melbourne, victoria]",[]
7,{'abstract': '<jats:p>Nonresonant manipulation...,10.1126/science.1072245,"[{'given': 'Thilo M.', 'family': 'Brill', 'seq...",8,"[[{'name': 'Schlumberger-Doll Research, 36 Old...",8,"[Schlumberger-Doll Research, 36 Old Quarry Roa...",1,"[SchlumbergerDoll Research, 36 Old Quarry Road...","[SchlumbergerDoll Research, 36 Old Quarry Road...","[schlumbergerdoll research, 36 old quarry road...",[]
11,{'URL': 'http://dx.doi.org/10.1161/01.cir.0000...,10.1161/01.cir.0000019680.58760.ac,"[{'given': 'Ruth', 'family': 'SoRelle', 'seque...",1,[[{'name': 'Circulation Newswriter'}]],1,[Circulation Newswriter],1,[Circulation Newswriter],[Circulation Newswriter],[circulation newswriter],[]
23,"{'URL': 'http://dx.doi.org/10.1021/op0255130',...",10.1021/op0255130,"[{'given': 'Bryan H.', 'family': 'Landis', 'se...",4,[[{'name': 'Bioprocess and Formulation Technol...,4,"[Bioprocess and Formulation Technology, Pharma...",1,"[Bioprocess and Formulation Technology, Pharma...","[Bioprocess and Formulation Technology, Pharma...","[bioprocess and formulation technology, pharma...",[]
...,...,...,...,...,...,...,...,...,...,...,...,...
7049,{'abstract': '<jats:p> Avenir d’enfants est né...,10.1177/1757975914520787,"[{'given': 'Lyse', 'family': 'Brunet', 'sequen...",1,[[{'name': 'Lyse Brunet était directrice génér...,1,[Lyse Brunet était directrice générale d’Aveni...,1,[Lyse Brunet était directrice générale dAvenir...,[Lyse Brunet était directrice générale dAvenir...,[lyse brunet était directrice générale davenir...,[]
7050,"{'URL': 'http://dx.doi.org/10.1111/ajr.12111',...",10.1111/ajr.12111,"[{'given': 'Gordon', 'family': 'Gregory', 'seq...",1,[[{'name': 'National Rural Health Alliance; De...,1,[National Rural Health Alliance; Deakin West A...,1,[National Rural Health Alliance Deakin West AC...,[National Rural Health Alliance Deakin West AC...,[national rural health alliance deakin west ac...,[]
7089,"{'URL': 'http://dx.doi.org/10.1002/etc.2609', ...",10.1002/etc.2609,"[{'given': 'Thomas W.', 'family': 'Custer', 's...",5,[[{'name': 'US Geological Survey; La Crosse Wi...,5,"[US Geological Survey; Madison Wisconsin USA, ...",3,"[US Geological Survey Madison Wisconsin USA, U...","[US Geological Survey Madison Wisconsin USA, U...","[us geological survey madison wisconsin usa, u...",[]
7227,{'URL': 'http://dx.doi.org/10.1111/npqu.11441'...,10.1111/npqu.11441,"[{'given': 'ELIF', 'family': 'SHAFAK', 'sequen...",1,[[{'name': 'Turkish novelist and author The Ba...,1,[Turkish novelist and author The Bastard of Is...,1,[Turkish novelist and author Bastard of Istanb...,[Turkish novelist and author Bastard of Istanb...,[turkish novelist and author bastard of istanbul],[]


In [97]:
len(affilDF)  - len(notInList)

6891

In [98]:
len(notInList)

406

# affilDF1 ['DOI', 'affiliations', 'Dictionary','uniqueAff4', 'uniqueAff2','# authors','# uniqueAff']

In [99]:
affilDF1 = affilDF[['DOI', 'affiliations', 'Dictionary','uniqueAff4', 'uniqueAff2','# authors','# uniqueAff']]

## new column: category based on the labels 

In [100]:
category = [', '.join(list(set([x[1] for x in affilDF1['Dictionary'].iloc[i]]))) for i in range(len(affilDF1))]
    


In [101]:
affilDF1 = affilDF1.copy()

In [102]:
affilDF1.loc[:, 'category'] = category


### new label: rest

In [103]:
for i in range(len(affilDF1)):
    if len(affilDF1['category'].iloc[i]) == 0:
        affilDF1.iloc[i, affilDF1.columns.get_loc('category')] = 'Rest'


In [104]:
affiliationsSimple = [list(set([x[0] for x in affilDF1['Dictionary'].iloc[i]])) for i in range(len(affilDF1))]

In [105]:
city_names = ["Aberdeen", "Abilene", "Akron", "Albany", "Albuquerque", "Alexandria", "Allentown", "Amarillo", "Anaheim", "Anchorage", "Ann Arbor", "Antioch", "Apple Valley", "Appleton", "Arlington", "Arvada", "Asheville", "Athens", "Atlanta", "Atlantic City", "Augusta", "Aurora", "Austin", "Bakersfield", "Baltimore", "Barnstable", "Baton Rouge", "Beaumont", "Bel Air", "Bellevue", "Berkeley", "Bethlehem", "Billings", "Birmingham", "Bloomington", "Boise", "Boise City", "Bonita Springs", "Boston", "Boulder", "Bradenton", "Bremerton", "Bridgeport", "Brighton", "Brownsville", "Bryan", "Buffalo", "Burbank", "Burlington", "Cambridge", "Canton", "Cape Coral", "Carrollton", "Cary", "Cathedral City", "Cedar Rapids", "Champaign", "Chandler", "Charleston", "Charlotte", "Chattanooga", "Chesapeake", "Chicago", "Chula Vista", "Cincinnati", "Clarke County", "Clarksville", "Clearwater", "Cleveland", "College Station", "Colorado Springs", "Columbia", "Columbus", "Concord", "Coral Springs", "Corona", "Corpus Christi", "Costa Mesa", "Dallas", "Daly City", "Danbury", "Davenport", "Davidson County", "Dayton", "Daytona Beach", "Deltona", "Denton", "Denver", "Des Moines", "Detroit", "Downey", "Duluth", "Durham", "El Monte", "El Paso", "Elizabeth", "Elk Grove", "Elkhart", "Erie", "Escondido", "Eugene", "Evansville", "Fairfield", "Fargo", "Fayetteville", "Fitchburg", "Flint", "Fontana", "Fort Collins", "Fort Lauderdale", "Fort Smith", "Fort Walton Beach", "Fort Wayne", "Fort Worth", "Frederick", "Fremont", "Fresno", "Fullerton", "Gainesville", "Garden Grove", "Garland", "Gastonia", "Gilbert", "Glendale", "Grand Prairie", "Grand Rapids", "Grayslake", "Green Bay", "GreenBay", "Greensboro", "Greenville", "Gulfport-Biloxi", "Hagerstown", "Hampton", "Harlingen", "Harrisburg", "Hartford", "Havre de Grace", "Hayward", "Hemet", "Henderson", "Hesperia", "Hialeah", "Hickory", "High Point", "Hollywood", "Honolulu", "Houma", "Houston", "Howell", "Huntington", "Huntington Beach", "Huntsville", "Independence", "Indianapolis", "Inglewood", "Irvine", "Irving", "Jackson", "Jacksonville", "Jefferson", "Jersey City", "Johnson City", "Joliet", "Kailua", "Kalamazoo", "Kaneohe", "Kansas City", "Kennewick", "Kenosha", "Killeen", "Kissimmee", "Knoxville", "Lacey", "Lafayette", "Lake Charles", "Lakeland", "Lakewood", "Lancaster", "Lansing", "Laredo", "Las Cruces", "Las Vegas", "Layton", "Leominster", "Lewisville", "Lexington", "Lincoln", "Little Rock", "Long Beach", "Lorain", "Los Angeles", "Louisville", "Lowell", "Lubbock", "Macon", "Madison", "Manchester", "Marina", "Marysville", "McAllen", "McHenry", "Medford", "Melbourne", "Memphis", "Merced", "Mesa", "Mesquite", "Miami", "Milwaukee", "Minneapolis", "Miramar", "Mission Viejo", "Mobile", "Modesto", "Monroe", "Monterey", "Montgomery", "Moreno Valley", "Murfreesboro", "Murrieta", "Muskegon", "Myrtle Beach", "Naperville", "Naples", "Nashua", "Nashville", "New Bedford", "New Haven", "New London", "New Orleans", "New York", "New York City", "Newark", "Newburgh", "Newport News", "Norfolk", "Normal", "Norman", "North Charleston", "North Las Vegas", "North Port", "Norwalk", "Norwich", "Oakland", "Ocala", "Oceanside", "Odessa", "Ogden", "Oklahoma City", "Olathe", "Olympia", "Omaha", "Ontario", "Orange", "Orem", "Orlando", "Overland Park", "Oxnard", "Palm Bay", "Palm Springs", "Palmdale", "Panama City", "Pasadena", "Paterson", "Pembroke Pines", "Pensacola", "Peoria", "Philadelphia", "Phoenix", "Pittsburgh", "Plano", "Pomona", "Pompano Beach", "Port Arthur", "Port Orange", "Port Saint Lucie", "Port St. Lucie", "Portland", "Portsmouth", "Poughkeepsie", "Providence", "Provo", "Pueblo", "Punta Gorda", "Racine", "Raleigh", "Rancho Cucamonga", "Reading", "Redding", "Reno", "Richland", "Richmond", "Richmond County", "Riverside", "Roanoke", "Rochester", "Rockford", "Roseville", "Round Lake Beach", "Sacramento", "Saginaw", "Saint Louis", "Saint Paul", "Saint Petersburg", "Salem", "Salinas", "Salt Lake City", "San Antonio", "San Bernardino", "San Buenaventura", "San Diego", "San Francisco", "San Jose", "Santa Ana", "Santa Barbara", "Santa Clara", "Santa Clarita", "Santa Cruz", "Santa Maria", "Santa Rosa", "Sarasota", "Savannah", "Scottsdale", "Scranton", "Seaside", "Seattle", "Sebastian", "Shreveport", "Simi Valley", "Sioux City", "Sioux Falls", "South Bend", "South Lyon", "Spartanburg", "Spokane", "Springdale", "Springfield", "St. Louis", "St. Paul", "St. Petersburg", "Stamford", "Sterling Heights", "Stockton", "Sunnyvale", "Syracuse", "Tacoma", "Tallahassee", "Tampa", "Temecula", "Tempe", "Thornton", "Thousand Oaks", "Toledo", "Topeka", "Torrance", "Trenton", "Tucson", "Tulsa", "Tuscaloosa", "Tyler", "Utica", "Vallejo", "Vancouver", "Vero Beach", "Victorville", "Virginia Beach", "Visalia", "Waco", "Warren", "Washington", "Waterbury", "Waterloo", "West Covina", "West Valley City", "Westminster", "Wichita", "Wilmington", "Winston", "Winter Haven", "Worcester", "Yakima", "Yonkers", "York", "Youngstown"]

city_names = [x.lower() for x in city_names]

In [106]:
new_affiliations = []
for x in affiliationsSimple:
    new_x = []
    for elem in x:
        if elem not in city_names:
            new_x.append(elem)
    new_affiliations.append(new_x)



In [107]:
affilDF1.loc[:,'affilSimple'] = new_affiliations

In [108]:
len(affilDF1[affilDF1['category'] == 'Rest'])

406

# UNIVS & LABS

In [109]:
univLabs = [i for i in range(len(affilDF1)) if 'Laboratory' in affilDF1['category'].iloc[i] 
            or 'Univ/Inst' in  affilDF1['category'].iloc[i]]

In [110]:

univLabsDF = affilDF1.iloc[univLabs].copy()

In [111]:
univLabsDF.reset_index(inplace = True)

In [112]:
univLabsDF.drop(columns = ['index'], inplace = True)

# Load files from openAIRE

In [115]:
with open('dixOpenAIRE_Alletc.pkl', 'rb') as f:
    dixOpenAIRE_Alletc = pickle.load(f)


## clean/modify the files

In [116]:
dixOpenAIRE_Alletc1 =  {k.replace(',', ''): v for k, v in dixOpenAIRE_Alletc.items()}

In [117]:
del dixOpenAIRE_Alletc1['laboratory']

In [118]:
del dixOpenAIRE_Alletc1['university hospital']

# MATCHINGS

# helper functions

In [119]:
def is_contained(s, w):
    words = s.split()  # Split the string 's' into a list of words
    for word in words:
        if word not in w:  # If a word from 's' is not found in 'w'
            return False  # Return False immediately
    return True  # If all words from 's' are found in 'w', return True

In [126]:
'dfsdfdsfsf university sffgsdg'.count('university')

1

In [132]:
l1 = ['university og gfdg', 'univ sdfdsfs dsfsdf', 'sdfdfsdf university', 'sdfdsf']

In [135]:
numUniv = sum([l1[i].count('univ') for i in range(len(l1))])
numUniv


3

In [146]:
is_contained("univ", 'university of athens')

True

In [241]:
def bestSimScore3(l1, l2, l3):
    vectorizer = CountVectorizer()
    # l1 = ['Unique affiliations'].iloc[i]]
    # l2 =  ['Matched openAIRE names'].iloc[i])]
    # l3 = [pairs.iloc[i]]
    numUniv = sum([(l1[i].lower()).count('univ') for i in range(len(l1))])
    result = []
    for i in range(len(l1)):
        best = [] 
        s = l1[i]
       # s_contains_university = is_contained("university", s.lower())  
        
        if not is_contained("univ", s.lower()):
            continue  # Skip if s does not contain "university" or "univ"
        
    
        for j in range(len(l2)):
            x = l2[j]  
              
            if l3[j][2] >=0.99 and (is_contained("univ", x.lower()) or is_contained("college", x.lower()) or  is_contained("center", x.lower())): # If the similarity score of a pair (s,x) was 1, we store it to results list
                result.append([l2[j], 1])
                
            else:
        #        x_contains_university = is_contained("university", x.lower())
                if not is_contained("univ", x.lower()):
                    continue  # Skip if x does not contain "university" or "univ"
                
            
                x_vector = vectorizer.fit_transform([x]).toarray() #Else we compute the similarity of s with the original affiiation name
                s_vector = vectorizer.transform([s]).toarray()

                # Compute similarity between the vectors
                similarity = cosine_similarity(x_vector, s_vector)[0][0]

                best.append([l2[j], similarity])
        
        if best:
            max_score = max(best, key=lambda x: x[1])[1]
            max_results = [(x[0], x[1]) for x in best if x[1] == max_score]
            if len(max_results) > 1:
                max_results.sort(key=lambda x: (l2.index(x[0]), -x[1]), reverse=False)
                result.append(max_results[-1])
            else:
                result.append(max_results[0])
                
    univ_list = []
    other_list = []
    
    for r in result:
        if is_contained('univ',r[0]):
            univ_list.append(r)
        else:
            other_list.append(r)
            #else:
            #   result.append((None, 0.0))
    if len(univ_list)> numUniv:
        result = univ_list[:numUniv] + other_list
                
    return result

## main map

In [242]:
def Doi_Pid_v1i(m, DF, dixOpenAIRE, simU, simG):
    
    """
    Matches affiliations in DataFrame 'DF' with names from dictionary 'dixOpenAIRE' and their PIDs based on similarity scores.

    Args:
        m (int): The number of DOIs to check.
        DF (DataFrame): The input DataFrame containing affiliation data.
        dixOpenAIRE (dict): A dictionary of names from OpenAIRE.
        simU (float): Similarity threshold for universities.
        simG (float): Similarity threshold for non-universities.

    Returns:
        DataFrame: The final DataFrame with matched affiliations and their corresponding similarity scores.
    """
    
    lnamelist = list(dixOpenAIRE.keys())
    dix = {}
    deiktes = []
    vectorizer = CountVectorizer()
    similarity_ab = []
    pairs = []

    for k in range(m):
        similar_k = []
        pairs_k = []

        for s in DF['affilSimple'].iloc[k]:
            if s in lnamelist:
                deiktes.append(k)
                similarity = 1
                similar_k.append(similarity)
                
                pairs_k.append((s,s,similarity))
                if k not in dix:
                    dix[k] = [s]
                else:
                    dix[k].append(s)
            else:

                for x in lnamelist:
                    
                    if  is_contained(s, x):
                        x_vector = vectorizer.fit_transform([x]).toarray()
                        s_vector = vectorizer.transform([s]).toarray()

                        # Compute similarity between the vectors
                        similarity = cosine_similarity(x_vector, s_vector)[0][0]
                        if similarity > min(simU, simG):
                            if (is_contained('univ', s) and is_contained('univ', x)) and similarity> simU:
                                similar_k.append(similarity)
                                deiktes.append(k)
                                pairs_k.append((s,x,similarity))


                                if k not in dix:
                                    dix[k] = [x]
                                else:
                                    dix[k].append(x)
                            elif (not is_contained('univ', s) and not is_contained('univ', x)) and similarity> simG:
                                similar_k.append(similarity)
                                deiktes.append(k)
                                pairs_k.append((s,x,similarity))


                                if k not in dix:
                                    dix[k] = [x]
                                else:
                                    dix[k].append(x)
                    elif is_contained(x, s):
                        if (is_contained('univ', s) and is_contained('univ', x)):

                            if 'and' in s:
                                list_s = s.split(' and ')
                                for t in list_s:
                                    if is_contained(x, t) and is_contained('univ', t):
                                        t_vector = vectorizer.fit_transform([t]).toarray()
                                        x_vector = vectorizer.transform([x]).toarray()

                            # Compute similarity between the vectors
                                        similarity = cosine_similarity(t_vector, x_vector)[0][0]
                                        if similarity > simU:
                                            similar_k.append(similarity)
                                            deiktes.append(k)
                                            pairs_k.append((s,x,similarity))


                                            if k not in dix:
                                                dix[k] = [x]
                                            else:
                                                dix[k].append(x)
                            
                            else: 
                                s_vector = vectorizer.fit_transform([s]).toarray()
                                x_vector = vectorizer.transform([x]).toarray()

                                # Compute similarity between the vectors
                                similarity = cosine_similarity(s_vector, x_vector)[0][0]
                                if similarity > 0.81: #max(0.82,sim):
                                    similar_k.append(similarity)
                                    deiktes.append(k)
                                    pairs_k.append((s,x,similarity))


                                    if k not in dix:
                                        dix[k] = [x]
                                    else:
                                        dix[k].append(x)
                        elif not is_contained('univ', s) and not is_contained('univ', x):
                            if 'and' in s:
                                list_s = s.split(' and ')
                                for t in list_s:
                                    if is_contained(x, t):
                                        t_vector = vectorizer.fit_transform([t]).toarray()
                                        x_vector = vectorizer.transform([x]).toarray()

                            # Compute similarity between the vectors
                                        similarity = cosine_similarity(t_vector, x_vector)[0][0]
                                        if similarity > simG:
                                            similar_k.append(similarity)
                                            deiktes.append(k)
                                            pairs_k.append((s,x,similarity))


                                            if k not in dix:
                                                dix[k] = [x]
                                            else:
                                                dix[k].append(x)
                            
                            else: 
                                s_vector = vectorizer.fit_transform([s]).toarray()
                                x_vector = vectorizer.transform([x]).toarray()

                                # Compute similarity between the vectors
                                similarity = cosine_similarity(s_vector, x_vector)[0][0]
                                if similarity > simG: #max(0.82,sim):
                                    similar_k.append(similarity)
                                    deiktes.append(k)
                                    pairs_k.append((s,x,similarity))


                                    if k not in dix:
                                        dix[k] = [x]
                                    else:
                                        dix[k].append(x)
                            
        similarity_ab.append(similar_k)   
        similarity_ab = [lst for lst in similarity_ab if lst != []]
        pairs.append(pairs_k)
        
    perc = 100*len(dix)/m
    
    dixDoiAff = {DF['DOI'].iloc[key]: value for key, value in dix.items()}
    dixDoiPid = {key : [dixOpenAIRE[x] for x in value if x in  lnamelist] for key , value in dixDoiAff.items()}
    
    dixDoiPid = {key : [dixOpenAIRE[x] for x in value] for key , value in dixDoiAff.items()}
    
## Define the new Dataframe
    
    doiPidDF = pd.DataFrame()
    doiPidDF['DOI'] = list(dixDoiPid.keys())
    doiPidDF['Affiliations'] = list(DF['affiliations'].iloc[list(set(deiktes))])

    doiPidDF['Unique affiliations'] = list(DF['uniqueAff2'].iloc[list(set(deiktes))])
    
    doiPidDF['# Authors'] = list(DF['# authors'].iloc[list(set(deiktes))])


    doiPidDF['# Unique affiliations'] = list(DF['# uniqueAff'].iloc[list(set(deiktes))])

    doiPidDF['Candidates for matching'] = list(DF['affilSimple'].iloc[list(set(deiktes))])

    doiPidDF['Matched openAIRE names'] = list(dix.values())
    doiPidDF['# Matched PIDs'] = [len(list(dix.values())[i]) for i in range(len(list(dix.values())))]

    doiPidDF['Similarity score'] = similarity_ab
    perfectSim = [[1 if num >= 1 else 0 for num in inner_list] for inner_list in similarity_ab]

    doiPidDF['Perfect match'] = perfectSim
    perfectSimSum = [sum(x) for x in perfectSim]
    doiPidDF['Perfect sum'] = perfectSimSum
    Pairs = [lst for lst in pairs if lst]
    doiPidDF['Pairs'] = Pairs
    


## Correct the matchings
    
    needCheck = [i for i  in range(len(doiPidDF)) if doiPidDF['# Matched PIDs'].iloc[i] - max(doiPidDF['# Authors'].iloc[i],doiPidDF['# Unique affiliations'].iloc[i]) >0]
    ready = [i for i in range(len(doiPidDF)) if i not in needCheck]
   
    best = [ bestSimScore3(doiPidDF['Unique affiliations'].iloc[i], doiPidDF['Matched openAIRE names'].iloc[i], doiPidDF['Pairs'].iloc[i]) for i in needCheck]
    best_o = []
    best_s = []
    
    for x in best:
        best_o.append([x[i][0]  for i in range(len(x))])
        best_s.append([round(x[i][1],2)  for i in range(len(x))])
    numMathced = [len(best_s[i]) for i in range(len(needCheck))]
    

    
    dfFinal0 = (doiPidDF.iloc[ready]).copy()
    dfFinal0['index'] = ready
    
    dfFinal2 = (doiPidDF.iloc[needCheck]).copy()
    dfFinal2['index'] = needCheck
    dfFinal2['Matched openAIRE names'] = best_o
    dfFinal2['Similarity score'] = best_s
    dfFinal2['# Matched PIDs'] = numMathced
    
    finalDF =  pd.concat([dfFinal0, dfFinal2])
    finalDF.set_index('index', inplace=True)
    finalDF.sort_values('index', ascending=True, inplace = True)
    

    
    
    
    return [perc,finalDF, needCheck, doiPidDF, pairs]
    
    


In [243]:
result_v1i = Doi_Pid_v1i(150, univLabsDF, dixOpenAIRE_Alletc1, 0.7,0.82)

# HTML

In [244]:
mydf = result_v1i[1]
myDF = mydf[["Unique affiliations",'# Authors','# Unique affiliations', '# Matched PIDs','Candidates for matching', 'Matched openAIRE names', 'Similarity score', 'Perfect match', 'Pairs']]

In [245]:
app = dp.App(dp.Page(title="matching", blocks= [dp.Text('Matched '+str(result_v1i[0])+'%'), dp.Text('DOIs with aff. in Univ/Inst, Laboratories'), dp.Table(myDF),  
                                               
                     ]
             ))
    


   
app.save(path="matching4.html", open=True)

App saved to ./matching4.html